In [2]:
# OPTIONAL: Load the "autoreload" extension so that code can change.
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded.
%autoreload 2

# Смена рабочей папки.
%cd ..

d:\coding\text-proc-ml


d:\coding\text-proc-ml\.venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


2.4.3 Tokenization

In [5]:
import re
import sys

In [32]:
TOKENIZE_RE = re.compile(r"[а-яё]+|-?\d*[.,]?\d+|\S", re.I)
# -?\d*[.,]?\d+


def tokenize(txt):
    return TOKENIZE_RE.findall(txt)

In [38]:
text_corpus = [
    "Тестовое сообщение",
    "Мама мыла раму.",
    "Контактный телефон: 123123.",
    "Что-нибудь надо придумать.",
    "Значение числа Е=2.7182.",
    "Демон123, как тебя зовут в реале?",
    "-1-.15=-1.15",
    "- 1 - .15 = -1.15",
    "Какого ;%:?* тут происходит?",
]
for text_value in text_corpus:
    print(" ".join(tokenize(text_value.strip().lower())))

тестовое сообщение
мама мыла раму .
контактный телефон : 123123 .
что - нибудь надо придумать .
значение числа е = 2.7182 .
демон 123 , как тебя зовут в реале ?
-1 -.15 = -1.15
- 1 - .15 = -1.15
какого ; % : ? * тут происходит ?


# 2.4.5 Вектор весов

In [3]:
import re
from dlnlputils.data import (
    tokenize_text_simple_regex,
    tokenize_corpus,
    build_vocabulary,
    vectorize_texts,
    SparseFeaturesDataset,
)

In [4]:
text_corpus_for_w = [
    "Казнить нельзя, помиловать. Нельзя наказывать.",
    "Казнить, нельзя помиловать. Нельзя освободить.",
    "Нельзя не помиловать.",
    "Обязательно освободить.",
]

In [5]:
corpus_tokenized = tokenize_corpus(text_corpus_for_w)

In [10]:
corpus_tokenized

[['казнить', 'нельзя', 'помиловать', 'нельзя', 'наказывать'],
 ['казнить', 'нельзя', 'помиловать', 'нельзя', 'освободить'],
 ['нельзя', 'не', 'помиловать'],
 ['обязательно', 'освободить']]

In [7]:
MAX_DF = 1
MIN_COUNT = 0
vocabulary, word_doc_freq = build_vocabulary(
    corpus_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT
)
UNIQUE_WORDS_N = len(vocabulary)
print("Количество уникальных токенов", UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

Количество уникальных токенов 7
[('нельзя', 0), ('помиловать', 1), ('казнить', 2), ('освободить', 3), ('наказывать', 4), ('не', 5), ('обязательно', 6)]


In [14]:
vocabulary

{'нельзя': 0,
 'помиловать': 1,
 'казнить': 2,
 'освободить': 3,
 'наказывать': 4,
 'не': 5,
 'обязательно': 6}

In [8]:
word_df = [(word, word_doc_freq[i]) for i, (word, _) in enumerate(vocabulary.items())]

In [15]:
word_doc_freq

array([0.75, 0.75, 0.5 , 0.5 , 0.25, 0.25, 0.25], dtype=float32)

In [9]:
answer = sorted(word_df, key=lambda x: (x[1], x[0]))


answer_1 = []
answer_2 = []

for k, v in list(answer):
    answer_1.append(k)
    answer_2.append(str(v))

print(" ".join(answer_1))
print(" ".join(answer_2))

наказывать не обязательно казнить освободить нельзя помиловать
0.25 0.25 0.25 0.5 0.5 0.75 0.75


# Векторизация

In [12]:
corpus_tokenized

[['казнить', 'нельзя', 'помиловать', 'нельзя', 'наказывать'],
 ['казнить', 'нельзя', 'помиловать', 'нельзя', 'освободить'],
 ['нельзя', 'не', 'помиловать'],
 ['обязательно', 'освободить']]

In [16]:
VECTORIZATION_MODE = "tfidf"
vectors = vectorize_texts(
    corpus_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE
)

In [18]:
print(vectors)

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 13 stored elements and shape (4, 7)>
  Coords	Values
  (0, 0)	0.26666656136512756
  (0, 1)	0.13333328068256378
  (0, 2)	0.19999991357326508
  (0, 4)	0.39999982714653015
  (1, 0)	0.26666656136512756
  (1, 1)	0.13333328068256378
  (1, 2)	0.19999991357326508
  (1, 3)	0.19999991357326508
  (2, 0)	0.22222213447093964
  (2, 1)	0.22222213447093964
  (2, 5)	0.6666663885116577
  (3, 3)	0.4999997615814209
  (3, 6)	0.9999995231628418


In [70]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import numpy as np

corpus = [
    "Казнить нельзя, помиловать. Нельзя наказывать.",
    "Казнить, нельзя помиловать. Нельзя освободить.",
    "Нельзя не помиловать.",
    "Обязательно освободить.",
]

# Получаем счетчики слов
TF = CountVectorizer().fit_transform(corpus)

# Строим IDF.
vectorizer = TfidfVectorizer(smooth_idf=False, use_idf=True)
vectorizer.fit_transform(corpus)

## из IDF  в DF
word_doc_freq = 1 / np.exp(vectorizer.idf_ - 1)

# TF нормируем и сглаживаем логарифмом
TFIDF = np.log(TF.toarray() / TF.sum(axis=1) + 1) / word_doc_freq
# Масштабируем признаки
TFIDF = np.asarray(TFIDF)
scaledTFIDF = StandardScaler().fit_transform(TFIDF)

# Домножаем на np.sqrt((4-1)/4) для перевода из DDOF(0) в DDOF(1) для 4 текстов
scaledTFIDF *= np.sqrt(3 / 4)

# Вывод в порядке возрастания DF
for l in scaledTFIDF[:, np.argsort(word_doc_freq)]:
    print(" ".join(["%.2f" % d for d in l]))

1.50 -0.50 -0.50 0.87 -0.76 0.60 0.16
-0.50 -0.50 -0.50 0.87 0.18 0.60 0.16
-0.50 1.50 -0.50 -0.87 -0.76 0.29 1.04
-0.50 -0.50 1.50 -0.87 1.34 -1.48 -1.36
